# 第七章相关代码

# 4.3 卷积层的实现

In [4]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.util import im2col

In [7]:
# 例子
# 一个大矩阵包含1个矩阵，矩阵里有三个小矩阵，三个小矩阵分别是7行7列
x1 = np.random.rand(1, 3, 7, 7)
# print(x1)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # (9, 75)

(9, 75)


In [20]:
class Convolution:
    # 滤波器 偏置 步幅 填充

    def __init__(self, W, b, stride=1, pad=0):
            self.W = W
            self.b = b
            self.stride = stride
            self.pad = pad
            
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 滤波器的展开
        # reshape函数会自动计算-1维度上的元素个数，使多维数组的元素个数前后一致
        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
    
        return out

In [25]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
     # 展开(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
     # 最大值(2)
        out = np.max(col, axis=1)
    # 转换(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        return out